In [57]:
# imports

from catalog import Catalog
from course import Course

from bs4 import BeautifulSoup
import requests

In [58]:
# variable setup
catalog = Catalog()

catalog_url = 'http://catalogue.uci.edu/allcourses/engrmae/'
save_file_path = 'catalog.json'

In [70]:
page = requests.get(catalog_url)
soup = BeautifulSoup(page.content, 'html.parser')

courses = soup.find_all('div', class_='courseblock')

for course in courses:
    course_block_title = course.find('p', class_='courseblocktitle')
    course_block_desc = course.find('div', class_='courseblockdesc')

    course_block_title_text = course_block_title.text.split(sep='.')
    section, code = course_block_title_text[0].split(sep='\xa0')
    title = course_block_title_text[1].strip()
    units = course_block_title_text[2].split()[0]

    course_block_desc_children = course_block_desc.findChildren()
    description = course_block_desc_children[0].text

    for block in course_block_desc_children:
        text = block.text
        if 'Prerequisite' in text:
            if 'Corequisite' in text:
                text = text.split(sep='\n')
                corequisites = text[0].partition(' ')[2]
                prerequisites = text[1].partition(" ")[2]
            else:
                prerequisites = text.partition(" ")[2]
        elif 'Corequisite' in text:
            corequisite = text.partition(" ")[2]
        elif 'Design units' in text:
            design_units = text.strip("()").split()[-1]

    catalog.insert(Course(section, code, title, units, description, design_units=design_units, corequisites=corequisites, prerequisites=prerequisites))

TypeError: __init__() got an unexpected keyword argument 'design_units'

In [60]:
catalog.save_to_file(save_file_path)